In [1]:
import sys
import os

# Agregar la ruta a la raíz del proyecto al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [2]:
import tensorflow as tf

2025-05-06 11:03:55.648517: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-06 11:03:55.649119: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-06 11:03:55.652107: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-06 11:03:55.659192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746547435.671554  654176 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746547435.67

In [3]:
from prepare_data import LoadData
from prepare_data.methods.to_pandas import LoadWithPandas
from embeddings.storage.methods import ParquetStorageStrategy
from embeddings import GenerateEmbeddings
from embeddings.decoders.methods import GenerateEmbeddingsTF
from reduce_dimensions import UmapReduction


Cargamos el Dataset

In [4]:
load_strategy = LoadWithPandas('../data/data_test.csv', sep = ',')
data_loader = LoadData(load_strategy)

In [5]:
loaded_dataset = data_loader.load_data()
loaded_dataset

,subject,body,answer,type,queue,priority,language,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,Die Datenanalyse-Plattform brach unerwartet ab...,Ich werde Ihnen bei der Lösung des Problems he...,Incident,General Inquiry,low,de,Crash,Technical,Bug,Hardware,Resolution,Outage,Documentation,NaN
1,Customer Support Inquiry,Seeking information on digital strategies that...,We offer a variety of digital strategies and s...,Request,Customer Service,medium,en,Feedback,Sales,IT,Tech Support,NaN,NaN,NaN,NaN
2,Data Analytics for Investment,I am contacting you to request information on ...,I am here to assist you with data analytics to...,Request,Customer Service,medium,en,Technical,Product,Guidance,Documentation,Performance,Feature,NaN,NaN
3,Krankenhaus-Dienstleistung-Problem,Ein Medien-Daten-Sperrverhalten trat aufgrund ...,Zurück zur E-Mail-Beschwerde über den Sperrver...,Incident,Customer Service,high,de,Security,Breach,Login,Maintenance,Incident,Resolution,Feedback,NaN
4,Security,"Dear Customer Support, I am reaching out to in...","Dear [name], we take the security of medical d...",Request,Customer Service,medium,en,Security,Customer,Compliance,Breach,Documentation,Guidance,NaN,NaN
5,Concerns About Securing Medical Data on 2-in-1...,Inquiring about best practices for securing me...,Thank you for your concern regarding securing ...,Request,Technical Support,medium,en,Security,Product,Feature,IT,Tech Support,NaN,NaN,NaN
6,Ratung für Sicherung medizinischer Daten in Hu...,"Ratung, ob es möglich ist, Sicherung medizinis...","Wir freuen uns, Ihnen zu raten, Sicherung medi...",Request,Technical Support,medium,de,Backup,Security,IT,Tech Support,NaN,NaN,NaN,NaN
7,Problem with Integration,"The integration stopped working unexpectedly, ...",I will look into the problem and call you at <...,Problem,IT Support,high,en,Technical,Integration,Bug,Resolution,Outage,Documentation,NaN,NaN
8,Assistance Request,"Dear Customer Support, I am writing in regards...",I have received your report about the data blo...,Problem,Product Support,high,de,Technical,Bug,Security,Maintenance,Documentation,Resolution,NaN,NaN


Generamos los embeddings

In [6]:
embeddings_strategy = GenerateEmbeddingsTF(loaded_dataset, column = 'body', column_index='subject')
save_strategy = ParquetStorageStrategy('data.parquet')

E0000 00:00:1746547441.628051  654176 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1746547441.628359  654176 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
embedding_data = GenerateEmbeddings(embedding_strategy = embeddings_strategy, save_strategy = save_strategy)

In [8]:
embedding_data.generate_embeddings()

2025-05-06 11:04:04.724699: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


,id,embeddings
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,"[-0.0029212562, -0.04292097, 0.033873666, 0.02..."
1,Customer Support Inquiry,"[-0.013259977, -0.072572194, -0.011696001, 0.0..."
2,Data Analytics for Investment,"[-0.008241775, -0.06431195, 0.040233593, -0.05..."
3,Krankenhaus-Dienstleistung-Problem,"[-0.03922201, -0.038323674, -0.043939702, 0.03..."
4,Security,"[-0.06891327, 0.02468973, -0.032012098, 0.0057..."
5,Concerns About Securing Medical Data on 2-in-1...,"[-0.039212555, 0.020444715, -0.024272902, 0.01..."
6,Ratung für Sicherung medizinischer Daten in Hu...,"[0.0031486533, -0.0038348858, -0.029664455, 0...."
7,Problem with Integration,"[-0.041825414, -0.022260142, -0.021315293, -0...."
8,Assistance Request,"[-0.019702487, -0.03563253, -0.06562083, 0.018..."


In [9]:
embedding_data.save_embeddings(embedding_data.generate_embeddings())

In [10]:
data=embedding_data.generate_embeddings()
data

,id,embeddings
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,"[-0.0029212562, -0.04292097, 0.033873666, 0.02..."
1,Customer Support Inquiry,"[-0.013259977, -0.072572194, -0.011696001, 0.0..."
2,Data Analytics for Investment,"[-0.008241775, -0.06431195, 0.040233593, -0.05..."
3,Krankenhaus-Dienstleistung-Problem,"[-0.03922201, -0.038323674, -0.043939702, 0.03..."
4,Security,"[-0.06891327, 0.02468973, -0.032012098, 0.0057..."
5,Concerns About Securing Medical Data on 2-in-1...,"[-0.039212555, 0.020444715, -0.024272902, 0.01..."
6,Ratung für Sicherung medizinischer Daten in Hu...,"[0.0031486533, -0.0038348858, -0.029664455, 0...."
7,Problem with Integration,"[-0.041825414, -0.022260142, -0.021315293, -0...."
8,Assistance Request,"[-0.019702487, -0.03563253, -0.06562083, 0.018..."


In [11]:
reduccion = UmapReduction(column='embeddings', dataset=data, column_index = 'id')

In [12]:
reduccion.reduce_dimensions()

/home/carlos/.local/lib/python3.10/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


,id,dimensions
0,Unvorhergesehener Absturz der Datenanalyse-Pla...,"[7.87498, -5.8733244]"
1,Customer Support Inquiry,"[6.885969, -3.5558372]"
2,Data Analytics for Investment,"[7.5318494, -2.9058118]"
3,Krankenhaus-Dienstleistung-Problem,"[8.256192, -5.1214733]"
4,Security,"[7.7561126, -4.26596]"
5,Concerns About Securing Medical Data on 2-in-1...,"[8.390628, -3.7447228]"
6,Ratung für Sicherung medizinischer Daten in Hu...,"[8.845385, -5.6057334]"
7,Problem with Integration,"[6.906428, -4.994417]"
8,Assistance Request,"[6.407554, -4.382693]"
